# マスキングデータセットを作成

##　パッケージのインストール

In [90]:
!pip install --no-index --find-links=/dbfs/databricks/libraries/ langchain
!pip install --no-index --find-links=/dbfs/databricks/libraries/ langchain-openai
!pip install --no-index --find-links=/dbfs/databricks/libraries/ python-dotenv

Looking in links: /dbfs/databricks/libraries/
Looking in links: /dbfs/databricks/libraries/
Looking in links: /dbfs/databricks/libraries/


## ライブラリのインポート

In [91]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.pydantic_v1 import Field
import pandas as pd

## 環境変数の設定

In [ ]:
load_dotenv()

# AzureOpenAIのAPI KEY
azure_openai_api_key: str = ''
os.environ['AZURE_OPENAI_ENDPOINT'] = ''

## チャットオブジェクトを取得する関数

In [93]:
def get_llm(azure_deployment="gpt-4o-2024-11-20", api_version="2024-08-01-preview"):

    llm = AzureChatOpenAI(
        api_key=azure_openai_api_key,
        temperature=1,
        azure_deployment=azure_deployment,
        api_version=api_version,
    )
    return llm

## システムプロンプトの設定

In [143]:
generate_masking_content_system_prompt = """
【役割】
あなたは日本の個人情報をマスキングするためのデータセットを作成するデータ作成担当者です。
個人情報は、人の氏名、生年月日、会社名、住所、emailアドレス、電話番号です。

【背景】
個人情報をマスキングすることに特化したLLMを作成するために、マスキングされたデータセットが必要
あなたに個人情報をマスキングするためのデータセットを量産してもらいたい

【制約】
- 文章は日本語で、様々なバリエーションを持つ文章を生成して下さい。バリエーションが豊富であればあるほど良いです。自然な文章で長めの文章を生成して下さい。
- 自然な文章で1000文字以上で文章を生成して下さい。
- 文章の書き出しは可能な限り重複することがないように文章を作成してください。
- 文章は下記の【フォーマット】に従って必ず出力して下さい。「マスク前の文章」と「マスク後の文章」をワンセットで1件だけ生成して下さい。
- 生成した個人情報は、「マスク後の文章」で必ずマスキングしてください。元の個人情報が一切推測できないように完全にマスキングしてください。
- 必ず【必須で含める個人情報】を含めたマスキングデータを作成して下さい。
- 【任意で含める個人情報】を含めても良いです。自然な文章を作成するにあたり、【任意で含める個人情報】の内容が必要であれば含めて下さい。
- 【必須で含める個人情報】と【任意で含める個人情報】の内容は必ずマスキングして「マスク後の文章:」に出力して下さい。
- 【必須で含める個人情報】でも【任意で含める個人情報】でも、個人情報は【各個人情報のマスキング方法】に従ってマスキングするようにしてください。
- 文章に含まれる個人情報の数やその種類に関しても可能な限りバリエーションを持つようにしてください。
- 必須で含める個人情報は例にあるデータだけでなく可能な限り多様なバリエーションを含むようにしてください。

【必須で含める個人情報】
- カタカナで記載された個人の氏名をフルネーム（例: タナカタロウ、ヤマダハナコ、スズキイチロウなど、一般的に考えられる名前）
- 個人の名前は例に示された名前以外も考えて、生成すること。

【任意で含める個人情報】
- 会社名
- 日本の住所
- 電話番号
- 郵便番号
- emailアドレス
- 生年月日

【各個人情報のマスキング方法】
各情報は以下を参考にマスキングしてください。
- 人の氏名: <マスキング済みの氏名> （例: タナカタロウ → <マスキング済みの氏名>、ヤマダハナコ → <マスキング済みの氏名> など、元の氏名とは明らかに異なる、自然な日本人の氏名に見える文字列を全て<マスキング済みの氏名>にマスキング）
- 会社名: <マスキング済みの会社名>
- 日本の住所: <マスキング済みの住所>
- emailアドレス: <マスキング済みのemailアドレス>
- 電話番号: <マスキング済みの電話番号>
- 郵便番号: <マスキング済みの郵便番号>
- 生年月日: <マスキング済みの生年月日>

【フォーマット】
マスキング前の文章:IBM Japan Systems Engineering Co. Ltd.（ISE）の住所は東京都中央区日本橋箱崎町19番21号 MSH 日本橋箱崎ビルで、郵便番号は〒0123456でタナカタロウさんの職場の電話番号は0120000000です。
マスキング後の文章:<マスキング済みの会社名>の住所は<マスキング済みの住所>で、<マスキング済みの郵便番号>で<マスキング済みの氏名>さんの職場の電話番号は<マスキング済みの電話番号>です。
"""



set_before_masking_after_masking_system_prompt = """
【役割】
あなたはマスキング前の文章とマスキング後の文章を分けることAIアシスタントです。

【背景】
個人情報をマスキングすることに特化したLLMを作成するために、マスキングされたデータセットが必要です。
マスキング前の文章とマスキング後の文章を明確に分けてほしい。

【制約】
- 「マスク前の文章:」の文章を忠実にbefore_maskingフィールドに格納して下さい。
- 「マスク後の文章:」の文章を忠実にafter_maskingフィールドに格納して下さい。
- 「マスク前の文章」が存在する場合、必ず「マスク後の文章」を作成してください。
"""

## マスキング前と後の文章をテキスト抽出するクラス

In [144]:
class BeforeMaskingAfterMaskingModel(BaseModel):
    before_masking: str = Field(description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する')
    after_masking: str = Field(description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する')

    model_config = {
        "json_schema_extra": {
            "required": ["before_masking", "after_masking"]
        }
    }

In [145]:
def set_before_masking_after_masking(before_masking_after_masking_content):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                'system',
                set_before_masking_after_masking_system_prompt,
            ),
            ("human", "{input}"),
        ]
    )

    llm = get_llm()
    chain = prompt | llm.with_structured_output(BeforeMaskingAfterMaskingModel)
    output_qanda = chain.invoke({
        'input': before_masking_after_masking_content
        })
    
    return output_qanda

## マスキング用のデータを作成する関数

In [146]:
def generate_masking_content(system_prompt):
    try:
        llm = get_llm()
        user_query = "日本の個人情報をマスキングするためのデータセットを作成したいです。多彩で様々なバリエーションの文章を可能な限り長文生成して下さい。"

        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    system_prompt
                ),
                ("human", "{input}"),
            ]
        )

        chain = prompt | llm
        result = chain.invoke({
            'input': user_query
        })

        return result

    except Exception as e:
        print(f"タグ生成に失敗しました: {e}")
        return []

In [147]:
before_masking_after_masking_list = []
count = 100

for i in range(count):
    print(f"{i+1}件目のマスキングデータを生成開始")
    before_masking_after_masking_data = generate_masking_content(generate_masking_content_system_prompt)
    separate_before_masking_after_masking = set_before_masking_after_masking(before_masking_after_masking_data)

    row_data = {
        'before_mask': separate_before_masking_after_masking.before_masking,
        'after_mask': separate_before_masking_after_masking.after_masking
    }
    
    # リストに辞書を追加
    before_masking_after_masking_list.append(row_data)
    print(f"{i+1}件目のマスキングデータを生成完了")

1件目のマスキングデータを生成開始


c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の

1件目のマスキングデータを生成完了
2件目のマスキングデータを生成開始


c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の

2件目のマスキングデータを生成完了
3件目のマスキングデータを生成開始


...

98件目のマスキングデータを生成完了
99件目のマスキングデータを生成開始


c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の

99件目のマスキングデータを生成完了
100件目のマスキングデータを生成開始


c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\YukiYamanaka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の

100件目のマスキングデータを生成完了


In [148]:
before_masking_after_masking_list

[{'before_mask': '新しく開業した「ランチョン株式会社」は話題のスポットとなっています。特にその本社住所である東京都渋谷区恵比寿南2丁目8-5はおしゃれなエリアに位置し、多くの注目を集めています。同社の代表取締役であるヤマモトセイジさんの明るい性格と確かな経営手腕が成功を後押ししているようです。また、会社のメールアドレスであるinfo@lunchon.co.jpや問い合わせ電話番号03-1234-5678でも顧客からの連絡を受け付けています。さらに、ヤマモトセイジさんの個人的な生年月日は1975年5月18日とのことです。最近では、会社としての事業活動が益々活発化しており、特に「ランチョンお弁当宅配サービス」が大好評で、都内の各企業にも評判が広がっています。このサービスでは、暖かい食事を迅速に提供するとともに、環境にも配慮した取り組みを行っています。本社の郵便番号150-0022や上記のメールアドレスでも詳細な問い合わせが可能です。',
  'after_mask': '新しく開業した「<マスキング済みの会社名>」は話題のスポットとなっています。特にその本社住所である<マスキング済みの住所>はおしゃれなエリアに位置し、多くの注目を集めています。同社の代表取締役である<マスキング済みの氏名>さんの明るい性格と確かな経営手腕が成功を後押ししているようです。また、会社のメールアドレスである<マスキング済みのemailアドレス>や問い合わせ電話番号<マスキング済みの電話番号>でも顧客からの連絡を受け付けています。さらに、<マスキング済みの氏名>さんの個人的な生年月日は<マスキング済みの生年月日>とのことです。最近では、会社としての事業活動が益々活発化しており、特に「<マスキング済みの会社名>お弁当宅配サービス」が大好評で、都内の各企業にも評判が広がっています。このサービスでは、暖かい食事を迅速に提供するとともに、環境にも配慮した取り組みを行っています。本社の郵便番号<マスキング済みの郵便番号>や上記のメールアドレスでも詳細な問い合わせが可能です。'},
 {'before_mask': 'カンザキユウイチさんは、大手IT企業で働いており、現在の勤務先は「テクノロジーノート株式会社」です。会社の所在地は大阪府大阪市淀川区宮原4丁目8番1号であり、最寄り駅は新大阪

In [149]:
# 収集したデータからデータフレームを作成
df = pd.DataFrame(before_masking_after_masking_list)

# CSVに出力
df.to_csv('katakana_full_name.csv', index=False)